In [17]:
from PyPDF2 import PdfReader

from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI 

import tiktoken
import os

from APIkey import OPENAI_KEY

In [7]:
text = ''
reader = PdfReader('Plant diseases .pdf')
for page in reader.pages:
    text += page.extract_text()

In [10]:
text_splitter = CharacterTextSplitter(
separator = "\n",
chunk_size = 1000,
chunk_overlap = 200,
length_function = len
)

chunks = text_splitter.split_text(text)


In [66]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

vectorstore = FAISS.from_texts(texts = chunks , embedding = embeddings)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model='gpt-4',openai_api_key=OPENAI_KEY),
        retriever=vectorstore.as_retriever(),
        memory=memory
    )

In [29]:
query = 'Split all the main topics of the text into fully distinguishable and understandable sub topics in comma seperated words. Output only all the sub topics of the text in comma seperated words only.'

result = conversation_chain({'question':query})

In [30]:
topics =  result['answer'].split(',')
topics

['Seed and seedling diseases',
 ' smuts',
 ' wilts',
 ' root and crown rots',
 ' leaf blights',
 ' viruses',
 ' nematodes.']

In [34]:
# TOPIC SUPPOSED TO BE TAUGHT
topic = 'Nematodes' # input['topic']

# WAY OF TEACHING 
teacher = 'Pirate' # input['teacher']


query = f'Teach the subtopic : {topic} in the style of a {teacher}, in an understandable and fun way.'

output = conversation_chain({"question": query })


In [35]:
output['answer']

"Arr matey, gather 'round and let me spin ye a tale of these tiny creatures from the deep, the nematodes! These be not your typical sea monsters, but microscopic roundworms that roam the seven seas of soil, clutching their hollow needle-like mouthparts, the stylets, to puncture and feed on the hearty treasure of plant cells.\n\nNematodes, they be a crafty bunch, developing and feeding both inside and outside of a plant. And beware, sailors, they are easy to spread on your boots, tools, and ship equipment. \n\nTheir life journey, it be a strange one, starting as an egg, moving through several larval stages, and ending as an adult. Most of their young ones look just like small adults. But, should the seas turn rough and conditions become unfavorable, the females of some species transform into dormant, resistant forms called cysts. These cysts, they be as tough as a barnacle-covered hull, protecting their eggs for up to a decade!\n\nWhen a nematode attacks, the plant victim shows signs of

In [73]:
from flask import Flask, request, jsonify
from flask_cors import CORS

# Processed version of the Data!

app = Flask(__name__)

CORS(app)

def process(input):
    
    # TOPIC SUPPOSED TO BE TAUGHT
        topic = input['topic']

        # WAY OF TEACHING 
        teacher = input['teacher']

        if input['understood'] == True:
            query = f'Teach the subtopic : {topic} in the style of a {teacher}, in an understandable and fun way.  Also generate a fun quiz based on the text you have generated which the user will answer.'
        else: 
            query = f'I did not understand what you just said, could you repeat that in a more understandable and simple way?'
        output = conversation_chain({"question": query })
        return output['answer']
    
def processRun(input):
     query = f'''Here is some code, provide only output for it: 
     {input} 
 Even if you dont have the ability to execute code, just give the predicted output, DO NOT PROVIDE ANY UNNECESSARY TEXT OTHER THAN ONLY OUTPUT'''
     output = conversation_chain({"question": query })
     return output['answer']

def processHint(input):
    # STATICALLY MENTIONING PROBLEM STATEMENT BECAUSE NO TIME
    query = f'''Here is some code, the idea is to print the sum of 2 numbers, but I am stuck. Please help me : 
        {input} 
Please give me some hints on how to solve this    '''

    output = conversation_chain({'question':query})
    return output['answer']
        

@app.route('/', methods=['POST'])
def echo():
    try:

        data = request.get_json()
        
        # Check if 'info' key exists in the JSON data

        if 'info' in data:
            returnable = process(data['info'])
            return jsonify(returnable)
           
        
        
    except Exception as e:
        return e
    
@app.route('/get-topics')
def return_topics():
    return topics

@app.route('/run', methods=['POST'])
def run():
    data = request.get_json()

    returnable = processRun(data['info'])
    return jsonify(returnable)

@app.route('/hint', methods=['POST'])
def hint():
    data = request.get_json()

    returnable = processHint(data['info'])
    return jsonify(returnable)

if __name__ == '__main__':
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Oct/2023 19:14:50] "OPTIONS /hint HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 19:15:04] "POST /hint HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 19:15:31] "OPTIONS /hint HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 19:15:46] "POST /hint HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 19:16:15] "OPTIONS /hint HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 19:16:42] "POST /hint HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 19:18:27] "OPTIONS /hint HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 19:18:47] "POST /hint HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 19:19:42] "OPTIONS /hint HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 19:19:56] "POST /hint HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 20:04:16] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 20:04:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 20:05:52] "OPTIONS /run HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2023 20:05:58] "POST /run HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/